In [24]:
import pandas as pd

In [4]:
df = pd.read_csv("netflix_titles.csv")
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


Посмотрим на датасет и пропущенные значения

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [6]:
df.shape[0]

8807

In [7]:
df.dropna(inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5332 entries, 7 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       5332 non-null   object
 1   type          5332 non-null   object
 2   title         5332 non-null   object
 3   director      5332 non-null   object
 4   cast          5332 non-null   object
 5   country       5332 non-null   object
 6   date_added    5332 non-null   object
 7   release_year  5332 non-null   int64 
 8   rating        5332 non-null   object
 9   duration      5332 non-null   object
 10  listed_in     5332 non-null   object
 11  description   5332 non-null   object
dtypes: int64(1), object(11)
memory usage: 541.5+ KB


Посмотрим на все колонки и преобразуем по надобности

В некоторых колонках могут быть строковые значения через запятую. Такие колонки мы преобразуем в списки строк, с которыми будет проще взаимодействовать при помощи базы данных.

In [9]:
def parse_column(x):
    lst = x.split(', ')
    lst = [i.strip() for i in lst]
    return lst

In [10]:
df.type.unique()

array(['Movie', 'TV Show'], dtype=object)

Title

In [11]:
df.title.apply(parse_column).apply(len).unique()

array([1, 2, 3, 4], dtype=int64)

In [12]:
df.title = df.title.apply(parse_column)

Director

In [13]:
df.director.apply(parse_column).apply(len).unique()

array([ 1,  2,  9,  5,  3,  4, 11,  7, 13, 10, 12,  8], dtype=int64)

In [14]:
df.director = df.director.apply(parse_column)

Cast

In [15]:
df.cast.apply(parse_column).apply(len).unique()

array([ 8,  4, 11, 10,  9, 13, 21, 14,  7, 12,  6, 15, 17,  5, 36,  1,  3,
       16, 22, 19, 23,  2, 31, 18, 20, 26, 25, 41, 44, 28, 32, 38, 39, 34,
       24], dtype=int64)

In [16]:
df.cast = df.cast.apply(parse_column)

Country

In [17]:
df.country.apply(parse_column).apply(len).unique()

array([6, 1, 2, 3, 4, 5, 8, 7], dtype=int64)

In [18]:
df.country = df.country.apply(parse_column)

Listed in

In [19]:
df.listed_in.apply(parse_column).apply(len).unique()

array([3, 2, 1], dtype=int64)

In [20]:
df.listed_in = df.listed_in.apply(parse_column)

Rating

In [21]:
df.rating.unique()

array(['TV-MA', 'TV-14', 'PG-13', 'PG', 'R', 'TV-PG', 'G', 'TV-Y7',
       'TV-G', 'TV-Y', 'NC-17', 'NR', 'TV-Y7-FV', 'UR'], dtype=object)

В качестве бд будет MongoDB
поэтому перед загрузкой данные нужно преобразовать в json

In [22]:
records = df.to_dict('records')
records[0]

{'show_id': 's8',
 'type': 'Movie',
 'title': ['Sankofa'],
 'director': ['Haile Gerima'],
 'cast': ['Kofi Ghanaba',
  'Oyafunmike Ogunlano',
  'Alexandra Duah',
  'Nick Medley',
  'Mutabaruka',
  'Afemo Omilami',
  'Reggie Carter',
  'Mzuri'],
 'country': ['United States',
  'Ghana',
  'Burkina Faso',
  'United Kingdom',
  'Germany',
  'Ethiopia'],
 'date_added': 'September 24, 2021',
 'release_year': 1993,
 'rating': 'TV-MA',
 'duration': '125 min',
 'listed_in': ['Dramas', 'Independent Movies', 'International Movies'],
 'description': 'On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past.'}

In [29]:
!docker compose up -d

"docker" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [26]:
from pymongo import MongoClient

Создаем соединение с базой данных

In [27]:
client = MongoClient(
    'localhost',
    27017,
    username="admin",
    password="1q2w3e4r"
)

In [28]:
client.list_database_names()

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67851657e6759c6ed7746d4c, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

Подключаемся к базе данных (или создаем ее, если ее не было)

In [41]:
db = client['netflix']

In [42]:
collection = db.create_collection('movies')

Очистим коллекцию, на случай если там уже есть данные

In [43]:
collection.delete_many({})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [44]:
collection.insert_many(records)

InsertManyResult([ObjectId('67618451bab65752b70e4f5d'), ObjectId('67618451bab65752b70e4f5e'), ObjectId('67618451bab65752b70e4f5f'), ObjectId('67618451bab65752b70e4f60'), ObjectId('67618451bab65752b70e4f61'), ObjectId('67618451bab65752b70e4f62'), ObjectId('67618451bab65752b70e4f63'), ObjectId('67618451bab65752b70e4f64'), ObjectId('67618451bab65752b70e4f65'), ObjectId('67618451bab65752b70e4f66'), ObjectId('67618451bab65752b70e4f67'), ObjectId('67618451bab65752b70e4f68'), ObjectId('67618451bab65752b70e4f69'), ObjectId('67618451bab65752b70e4f6a'), ObjectId('67618451bab65752b70e4f6b'), ObjectId('67618451bab65752b70e4f6c'), ObjectId('67618451bab65752b70e4f6d'), ObjectId('67618451bab65752b70e4f6e'), ObjectId('67618451bab65752b70e4f6f'), ObjectId('67618451bab65752b70e4f70'), ObjectId('67618451bab65752b70e4f71'), ObjectId('67618451bab65752b70e4f72'), ObjectId('67618451bab65752b70e4f73'), ObjectId('67618451bab65752b70e4f74'), ObjectId('67618451bab65752b70e4f75'), ObjectId('67618451bab65752b70e4f

По ссылке http://localhost:8081 можно посмотреть на базу данных через веб-клиент

Ради интереса можем найти какие-нибудь записи из базы данных

In [47]:
tv_ma_rating_movies = [item for item in collection.find({"rating": "TV-MA"})]

In [48]:
len(tv_ma_rating_movies)

1822

In [49]:
tv_ma_rating_movies[0]

{'_id': ObjectId('67618451bab65752b70e4f5d'),
 'show_id': 's8',
 'type': 'Movie',
 'title': ['Sankofa'],
 'director': ['Haile Gerima'],
 'cast': ['Kofi Ghanaba',
  'Oyafunmike Ogunlano',
  'Alexandra Duah',
  'Nick Medley',
  'Mutabaruka',
  'Afemo Omilami',
  'Reggie Carter',
  'Mzuri'],
 'country': ['United States',
  'Ghana',
  'Burkina Faso',
  'United Kingdom',
  'Germany',
  'Ethiopia'],
 'date_added': 'September 24, 2021',
 'release_year': 1993,
 'rating': 'TV-MA',
 'duration': '125 min',
 'listed_in': ['Dramas', 'Independent Movies', 'International Movies'],
 'description': 'On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past.'}